## 

A prototype

#### Objective

The objective is to test whether 

Categorization of relations 
Changing the value of an attribute (e.g. onStatus) of one device/thing (e.g. smart-socket-1) may affect attributes of another device (e.g. smart-lamp-2)

We wanna make sure this doesn’t happen/users should get warned for altering a thing that has other things dependent on that 

```Cypher
MATCH (a:PowerSource {id: ‘smart-socket-1’})-[r:Powers]->(b:Appliance)
SET b.onStatus = False
RETURN b
```

```Cypher
MATCH (a:PowerSource {id: ‘smart-socket-1’})-[r:Powers]->(b:Appliance)
<detecting mechanism>
SET b.onStatus = False
RETURN b
```

#### Method

In this prototype, I will be using Neo4j's built-in example database, `Movie DBMS`.

One existing relationships in this db model could help with our prototyping process `Acted_in`

implies a dependency of an actor on a movie
analogous to the power source appliance relationship 

`onStatus`

##### Use cases to build model after

* power(on), appliance(off) --> wanting to switch off power

* power(off), appliance(on) --> wanting to switch on appliance

#### Steps
1. Modidying test data
* Add attributes to data
2. Building out the feature
* Write Cypher query that affects power/host's attributes
* Convert Cypher query into python function 
* Control statement to check power/appliance, determine whether to raise an alert

### Set up and test the connection

In [1]:
from py2neo import Graph

In [2]:
remote_url =  "neo4j://localhost:7687"
remote_user = "kaiser_zzk"
remote_pw =   "123465789"

In [3]:
def t1():
    graph = Graph(remote_url, auth=(remote_user, remote_pw))
    q = "match (r:Person) where r.name='Tom Hanks' return r"
    res = graph.run(q)

    for r in res:
        print(r)

In [4]:
t1()

Node('Person', born=1956, name='Tom Hanks')


### Modify `Movie` Nodes

Create and add an attribute that only accepts `True` or `False` as its values 

In [5]:
graph = Graph(remote_url, auth=(remote_user, remote_pw))

In [23]:
createPropertyTestStr = "MATCH (n:Movie) SET n.isTest2 = True"

In [24]:
def createPropertyTest():
    graph.run(createPropertyTestStr)

In [25]:
createPropertyTest()

There is currently no built-in way to enforce a range of values for a property in Neo4j (?)

In [37]:
def checkNodePropertyValue():
    q = "MATCH (n:Movie) WHERE n.title='When Harry Met Sally' RETURN n.released"
    num = graph.run(q)
    
    return num

In [38]:
checkNodePropertyValue()

n.released
1998


checking the value(s) of the related node(s) of the "root" node

In [49]:
def checkRelatedNodesPropertyValue():
    q = "MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = 'When Harry Met Sally' RETURN p.born"
    ages = graph.run(q)
    
    for a in ages:
        print(a)

In [50]:
checkRelatedNodesPropertyValue()

1956
1948
1949
1961
